<a href="https://colab.research.google.com/github/henry-bokyum-kim/NNStudy/blob/master/RL_NN_Impl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get install x11-utils

import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40)

import math
import glob
import io
import base64
from IPython.display import HTML

from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400,900),)
display.start()

Reading package lists... Done
Building dependency tree       
Reading state information... Done
x11-utils is already the newest version (7.7+3build1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1003'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1003'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

HIDDEN = 128
BATCH_SIZE = 32
PERCENTILE = 50

class Net(nn.Module):
  def __init__(self, obs_size, hidden, action_size):
    super(Net, self).__init__()
    self.net=nn.Sequential(
        nn.Linear(obs_size, hidden),
        nn.ReLU(),
        nn.Linear(hidden, action_size)
    )

  def forward(self, x):
    return self.net(x)

In [0]:
import copy

def get_batch(batch_size, env, net):
  batch = []
  reward = 0.0
  obs = env.reset()
  steps = []
  sm = nn.Softmax(dim=1)
  
  while True:
    obs_v = torch.FloatTensor([obs])
    action_v = sm(net(obs_v)).data.numpy()[0]
    action = np.random.choice(env.action_space.n, p=action_v)

    next_obs, rew, done, _ = env.step(action)
    reward += rew
    steps.append((obs,action))

    if done:
      batch.append((reward, steps))
      reward = 0.0
      next_obs = env.reset()
      steps = []
      if len(batch) == batch_size:
        yield batch
        batch=[]
    obs = next_obs

In [0]:
def filter_batch(batch):
  rews = list(map(lambda episode: episode[0],batch))
  rews_threshold = np.percentile(rews, PERCENTILE)
  rews_mean = np.mean(rews)

  ret_action = []
  ret_obs = []

  for scene in batch:
    if scene[0] < rews_threshold:
      continue
    ret_obs.extend(map(lambda step: step[0], scene[1]))
    ret_action.extend(map(lambda step: step[1], scene[1]))

  return torch.FloatTensor(ret_obs), torch.LongTensor(ret_action), rews_threshold, rews_mean

In [32]:
env = gym.wrappers.Monitor(gym.make("CartPole-v0"), "video", force=True, video_callable=lambda c:c%100 ==0)
net = Net(env.observation_space.shape[0], HIDDEN, env.action_space.n)
CELoss = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(), 0.01)

for i,batch in enumerate(get_batch(BATCH_SIZE, env, net)):
  obs, action, threshold, mean = filter_batch(batch)
  opt.zero_grad()

  action_out = net(obs)
  loss = CELoss(action_out, action)
  loss.backward()
  opt.step()
  print("%d, loss = %.4f, th = %.2f, mean = %.2f"%(i, loss, threshold, mean))
  if mean>199:
    print("Solved")
    break

NameError: ignored

In [22]:
batch[0]

(13.0,
 [(array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 0),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 0),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 0),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 1),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 0),
  (array([-0.03965737, -0.0360419 ,  0.0043652 , -0.02438641]), 0)])